### Libraries import

In [2]:
import tarfile
import nibabel as nib
import glob
import numpy as np 
import matplotlib.pyplot as plt
import os
import shutil
import json

data_dir = '/home/a-brazhnikov/nnUNet_raw_data_base/nnUNet_raw_data/Task602_brats_wT_3classes/'
imagesTr = os.path.join(data_dir, 'imagesTr')
imagesTr2 = os.path.join(data_dir, 'imagesTr2')
labelsTr = os.path.join(data_dir, 'labelsTr')

### Untar dataset

In [39]:
def untar_brats(path):
    tar = tarfile.open(path)
    tar.extractall('.')
    tar.close()
untar_brats('/home/a-brazhnikov/Brats2021/BraTS2021_Training_Data.tar')

### Preprocessing

In [19]:
# To be honest this cell was rewritten many times with different functionality

levels = ['t1','flair','t2','t1ce']
for path in glob.glob(os.path.join(imagesTr, '*')):
    
    fname = os.path.basename(path).split('.nii.gz')[0]
    '''
    # Skip if file with such name already exists
    if not os.path.exists(os.path.join(imagesTr, f'{fname}.nii.gz')):
        raw_image = [nib.load(os.path.join(path, f'{fname}_{level}.nii.gz')) for level in levels]

        # Save train file
        data = np.asarray([np.asanyarray(d.dataobj) for d in raw_image])
        image = nib.Nifti1Image(data, raw_image[0].affine, raw_image[0].header)
        nib.save(image, os.path.join(imagesTr, f'{fname}.nii.gz'))
    '''
    # labels 0,1,2,4 to 0,1,2,3
    label = nib.load(path)
    data = np.asarray(label.dataobj, dtype=np.uint8)
    #data[data==4] = 3
    image = nib.Nifti1Image(data, label.affine, label.header)
    nib.save(image, os.path.join(imagesTr2, f'{fname}_0004.nii.gz'))

In [ ]:
# 0000 - t1
# 0001 - t2
# 0002 - t1ce
# 0003 - flair
# to rename in bash:
# for file in *_t3*; do mv "$file" "${file/_t3.nii.gz/_0002.nii.gz}"; done

In [42]:
# dataset.json доделать под формат, потом переместить в nnunet_raw, сделать препроцессинг и начать обучение.

In [53]:
# labels to binary mask for the 5th layer model
# labels 0,1,2,3 to 0,1
data_dir = '/home/a-brazhnikov/nnUNet_raw_data_base/nnUNet_raw_data/Task602_brats_wT_3classes'
labelsTr = os.path.join(data_dir, 'labelsTr')

gl = glob.glob(os.path.join(labelsTr, '*'))
for i, path in enumerate(gl):
    label = nib.load(path)
    data = np.asanyarray(label.dataobj)
    print(np.unique(data))
    break
    
    data[data!=0] = 1
    image = nib.Nifti1Image(data, label.affine, label.header)
    nib.save(image, path)
    
    if (i+1)%20==0:
        print(f'{i+1}/{len(gl)}')

[0 1 2 3]


In [51]:
# Dataset json export
labelsTr = '/home/a-brazhnikov/nnUNet_raw_data_base/nnUNet_raw_data/Task602_brats_wT_3classes/labelsTr'

training = []
for path in glob.glob(os.path.join(labelsTr, '*')):
    fname = os.path.basename(path)
    training.append({
        'image': f'./imagesTr/{fname}',
        'label': f'./labelsTr/{fname}'
    })
print(len(training))

251


In [ ]:
jsonStr = json.dumps(training)
print(jsonStr)

In [22]:
len(training)

1251

In [49]:
os.path.basename(gl[0][:gl[0].rindex('_')]+'.nii.gz')

'BraTS2021_00353.nii.gz'

In [28]:
newdir = '/home/a-brazhnikov/nnUNet_raw_data_base/nnUNet_raw_data/Task602_brats_wT_3classes'
olddir = '/home/a-brazhnikov/nnUNet_raw_data_base/nnUNet_raw_data/Task600_brats'
newImg = os.path.join(newdir, 'imagesTr')
oldImg = os.path.join(olddir, 'imagesTr')

gl = glob.glob(os.path.join(newImg, '*'))
files = [os.path.basename(path).split('_')[1] for path in gl]

gl2 = glob.glob(os.path.join(oldImg, '*'))
files2 = [os.path.basename(path) for path in gl2]
for f in files2:
    if f.split('_')[1] in files:
        source = os.path.join(oldImg, f)
        shutil.copy(source, newImg)

In [ ]:
newLbl = os.path.join(newdir, 'labelsTr')
oldLbl = os.path.join(olddir, 'labelsTr')

gl2 = glob.glob(os.path.join(oldLbl, '*'))
files2 = [os.path.basename(path) for path in gl2]
for f in files2:
    if f.split('_')[1].split('.')[0] in files:
        source = os.path.join(oldLbl, f)
        shutil.copy(source, newLbl)

In [40]:
shutil.copy(olddir+'/dataset.json',newdir+'/dataset.json')

'/home/a-brazhnikov/nnUNet_raw_data_base/nnUNet_raw_data/Task602_brats_wT_3classes/dataset.json'

### Models evaluation

In [4]:
### Precision Recall
direct = '/home/a-brazhnikov/nnUNet_trained_models/nnUNet/3d_fullres/Task600_brats/nnUNetTrainerV2_100__nnUNetPlansv2.1/fold_0/validation_raw/'
fname = 'summary.json'
f = open(direct+fname)
data = json.load(f)

In [7]:
data['results'].keys()

dict_keys(['all', 'mean'])

In [9]:
for p in data['results']['mean']:
    if p>'0':
        res = data['results']['mean'][p]
        precission = res['Precision']
        recall = res['Recall']
        print(f'{p} {precission} {recall}')

1 0.8424811738416044 0.8195362818936061
2 0.8676067464795979 0.8734661128248886
3 0.8741236163717975 0.8888582788416072


In [5]:
direct = '/home/a-brazhnikov/nnUNet_trained_models/nnUNet/3d_fullres/Task601_brats_wholeTumor/nnUNetTrainerV2_100__nnUNetPlansv2.1/fold_0/validation_raw/'

fname = 'summary.json'
f = open(direct+fname)
data = json.load(f)

for p in data['results']['mean']:
    if p>'0':
        res = data['results']['mean'][p]
        precission = res['Precision']
        recall = res['Recall']
        print(f'{p} {precission} {recall}')

1 0.9468770881471237 0.9278853560254025
2 nan nan
3 nan nan


In [6]:
direct = '/home/a-brazhnikov/nnUNet_trained_models/nnUNet/3d_fullres/Task602_brats_wT_3classes/nnUNetTrainerV2_70__nnUNetPlansv2.1/fold_0/validation_raw/'

fname = 'summary.json'
f = open(direct+fname)
data = json.load(f)

for p in data['results']['mean']:
    if p>'0':
        res = data['results']['mean'][p]
        precission = res['Precision']
        recall = res['Recall']
        print(f'{p} {precission} {recall}')

1 0.8431371583551984 0.8679641710678553
2 0.8759775251836446 0.8677059991315136
3 0.9062500883588603 0.8982022370631257
